In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Dot, Add, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [5]:
# -------------------------------------------- Aggragated Emotions : literature_kaggle_ratings.csv ----------------------------------------------------- #
# Load the dataset
merged_books_ratings = pd.read_csv('/content/drive/MyDrive/ISR Project /literature_kaggle_ratings.csv')
print(merged_books_ratings.columns)

Index(['Unnamed: 0', 'Book_Id', 'Book', 'Author', 'Description', 'Genres',
       'Avg_Rating', 'Num_Ratings', 'URL', 'Aggregated Emotions',
       'Aggregated Des Emotions', 'ISBN', 'Book-Title', 'Book-Author',
       'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M',
       'Image-URL-L', 'User-ID', 'Book-Rating'],
      dtype='object')


In [6]:
# Preprocess and determine dominant mood
def determine_dominant_mood(aggregated_emotions):
    emotions_dict = literal_eval(aggregated_emotions)
    sorted_emotions = sorted(emotions_dict, key=lambda x: -x[1])
    #print(sorted_emotions[0][0])
    if len(sorted_emotions) >= 1 and sorted_emotions[0][0].strip().lower() == "happy" and ((sorted_emotions[0][1] - sorted_emotions[1][1]) <= 100):
        #print("reached this case")
        #print(sorted_emotions[1][0].strip().lower())
        #print(" ")
        return sorted_emotions[1][0].strip().lower()
    return sorted_emotions[0][0].strip().lower()

In [7]:
merged_books_ratings['Dominant_Mood'] = merged_books_ratings['Aggregated Emotions'].apply(determine_dominant_mood)

# Encoding user IDs, book IDs, and Dominant Moods
user_encoder = LabelEncoder()
book_encoder = LabelEncoder()
mood_encoder = LabelEncoder()

merged_books_ratings['user_id_encoded'] = user_encoder.fit_transform(merged_books_ratings['User-ID'])
merged_books_ratings['book_id_encoded'] = book_encoder.fit_transform(merged_books_ratings['ISBN'])
merged_books_ratings['mood_encoded'] = mood_encoder.fit_transform(merged_books_ratings['Dominant_Mood'])
print("Mood classes after fitting:", mood_encoder.classes_)

# Normalize ratings
merged_books_ratings['Book-Rating'] = merged_books_ratings['Book-Rating'].apply(lambda x: (x - 1) / 9)

# Split the data
train, test = train_test_split(merged_books_ratings, test_size=0.2, random_state=42)

Mood classes after fitting: ['anxious' 'attached' 'attracted' 'entitled' 'fearful' 'happy' 'powerless'
 'sad']


In [14]:
# Model architecture for collaborative filtering
def build_collaborative_filtering_model(num_users, num_books, embedding_size=15):
    # User and Book input layers
    user_input = Input(shape=(1,))
    book_input = Input(shape=(1,))

    # Embeddings
    user_embedding = Embedding(num_users, embedding_size, embeddings_regularizer=l2(1e-6))(user_input)
    book_embedding = Embedding(num_books, embedding_size, embeddings_regularizer=l2(1e-6))(book_input)

    # Flatten the embeddings
    user_vec = Flatten()(user_embedding)
    book_vec = Flatten()(book_embedding)

    # Dot product of user and book embeddings
    dot_product = Dot(axes=1)([user_vec, book_vec])

    # Add bias
    user_bias = Flatten()(Embedding(num_users, 1)(user_input))
    book_bias = Flatten()(Embedding(num_books, 1)(book_input))
    sum = Add()([dot_product, user_bias, book_bias])

    model = Model([user_input, book_input], sum)
    model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

    return model

In [15]:
# Build and train the model
model = build_collaborative_filtering_model(len(user_encoder.classes_), len(book_encoder.classes_))
model.fit([train['user_id_encoded'], train['book_id_encoded']], train['Book-Rating'], batch_size=64, epochs=5, validation_split=0.1)

Epoch 1/5
77/77 [==============================] - 1s 5ms/step - loss: 0.2711 - val_loss: 0.2608
Epoch 2/5
77/77 [==============================] - 0s 4ms/step - loss: 0.2483 - val_loss: 0.2490
Epoch 3/5
77/77 [==============================] - 0s 5ms/step - loss: 0.2258 - val_loss: 0.2397
Epoch 4/5
77/77 [==============================] - 1s 11ms/step - loss: 0.2008 - val_loss: 0.2321
Epoch 5/5
77/77 [==============================] - 1s 7ms/step - loss: 0.1715 - val_loss: 0.2259


In [16]:
# Function to recommend books based on user similarity and mood
def recommend_books(user_id, mood, top_n=5):
    user_idx = user_encoder.transform([user_id])
    mood_idx = mood_encoder.transform([mood.lower()])  # ensure lowercase for mood matching
    valid_books = merged_books_ratings[merged_books_ratings['mood_encoded'] == mood_idx[0]]['book_id_encoded'].unique()

    # Predict ratings for all books filtered by mood
    book_ratings = model.predict([np.array([user_idx[0]] * len(valid_books)), valid_books])
    top_books_idx = book_ratings.flatten().argsort()[-top_n:][::-1]
    recommended_books = book_encoder.inverse_transform(valid_books[top_books_idx])

    return recommended_books

In [17]:
# Test the recommendation function
test_user_id = user_encoder.classes_[0]  # Replace with a valid user ID
test_mood = 'happy'  # Replace with a valid mood
print("Recommended Books:", recommend_books(test_user_id, test_mood))

1/1 [==============================] - 0s 130ms/step
Recommended Books: ['0843941081' '0066620996' '0515131334' '0451201302']


In [18]:
# Evaluate the model
eval_loss = model.evaluate([test['user_id_encoded'], test['book_id_encoded']], test['Book-Rating'])
print(f'Evaluation Loss: {eval_loss}')

43/43 [==============================] - 0s 2ms/step - loss: 0.2249
Evaluation Loss: 0.22491636872291565


In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from PIL import Image
from io import BytesIO

# Assuming recommend_books() returns a list of recommended books
user_id = "26"
mood = "happy"
recommended_books = recommend_books(user_id, mood, top_n=5)

# Fetch the corresponding image URLs from the dataset
image_urls = merged_books_ratings.loc[merged_books_ratings['ISBN'].isin(recommended_books), 'Image-URL-S']
#print(image_urls)

# Display the recommendations with images
print(f"Recommendations for User ID {user_id} with mood '{mood}':")
for book, url in zip(recommended_books, image_urls):
    print(f" - {book}")
    print(url)
    try:
        # Check if the URL is valid
        if pd.notna(url) and requests.head(url).status_code == 200:
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            plt.figure()
            plt.imshow(img)
            plt.title(book)
            plt.axis('off')
            plt.show()
    except Exception as e:
        print(f"Error loading image for book {book}: {e}")
        continue  # Skip to the next book if there's an error
    print()  # Add space between different recommendations


1/1 [==============================] - 0s 39ms/step
Recommendations for User ID 26 with mood 'happy':
 - 0843941081
http://images.amazon.com/images/P/0066620996.01.THUMBZZZ.jpg

 - 0066620996
http://images.amazon.com/images/P/0066620996.01.THUMBZZZ.jpg

 - 0515131334
http://images.amazon.com/images/P/0066620996.01.THUMBZZZ.jpg

 - 0451201302
http://images.amazon.com/images/P/0066620996.01.THUMBZZZ.jpg

